In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 1000
# mpl.rcParams['legend.fontsize'] = 15
# params = {'legend.fontsize': 20,
#           'legend.handlelength': 2}
# plot.rcParams.update(params)

In [3]:
from algorithms.moo.nsga2 import NSGA2
from constraints.as_obj import ConstraintsAsObjective
from operators.survival.rank_and_crowding.classes import (
    RankAndCrowding,
    MyConstrRankAndCrowding,
    ParallelConstrRankAndCrowding,
    MyConstrRankAndCrowding2,
)
from optimize import minimize
from visualization.scatter import Scatter
from pymoo.problems import get_problem

In [4]:
core_moea1 = NSGA2(pop_size=500, survival=RankAndCrowding())
core_moea2 = NSGA2(pop_size=500, survival=MyConstrRankAndCrowding())
core_moea3 = NSGA2(pop_size=500, survival=ParallelConstrRankAndCrowding())
core_moea4 = NSGA2(pop_size=500, survival=MyConstrRankAndCrowding2())

In [5]:
import time


def performance_timer(func):
    """
    A decorator to measure and print the execution time of a function.
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Capture the start time
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # Capture the end time
        duration = end_time - start_time  # Calculate duration
        print(f"{func.__name__} executed in {duration:.6f} seconds")
        return result

    return wrapper


@performance_timer
def minimize_(*args, **kwargs):
    return minimize(*args, **kwargs)

In [6]:
from pymoo.indicators.gd import GD
from pymoo.indicators.gd_plus import GDPlus
from pymoo.indicators.igd import IGD
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV

metric_classes = {
    "Generational Distance (GD)": GD,
    "Generational Distance Plus (GD+)": GDPlus,
    "Inverted Generational Distance (IGD)": IGD,
    "Inverted Generational Distance Plus (IGD+)": IGDPlus,
    # "Hypervolume": HV,
}


def performance_indicators(pf, results):
    # Pre-configuration for each metric, handle special cases like HV here.
    pf_calculators = [
        (
            metric_class(ref_point=np.array([0, 0]))
            if metric_name == "Hypervolume"
            else metric_class(pf)
        )
        for metric_name, metric_class in metric_classes.items()
    ]

    def cal_acc_metric(sols):
        results_matrix = np.empty((len(sols), len(metric_classes)), dtype=object)

        for idx, sol in enumerate(sols):
            # Ensure `sol` is a 2D array with the correct shape [n_samples, n_features]
            sol = np.atleast_2d(sol)
            if sol.shape[0] == 1 and sol.shape[1] != len(pf):
                # This implies sol is a single sample with multiple features, which is the correct format
                pass
            elif sol.shape[1] == 1 and sol.shape[0] != len(pf):
                # This implies sol has the wrong orientation; it's many samples of a single feature
                sol = sol.T

            for metric_idx, calculator in enumerate(pf_calculators):
                try:
                    results_matrix[idx, metric_idx] = calculator(sol)
                except Exception as e:
                    # print(
                    #     f"Error calculating {list(metric_classes.keys())[metric_idx]}: {e}")
                    results_matrix[idx, metric_idx] = np.nan

        return [
            (np.mean(results_matrix[:, i]), np.median(results_matrix[:, i]))
            for i in range(len(pf_calculators))
        ]

    return [cal_acc_metric(rs) for rs in results]

In [7]:
from constraints.as_obj import CVRAsObjective, CDFAsObjective, CVRAsObjective2, CDFAsObjective2


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        # (ConstraintsAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CDFAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (ConstraintsAsObjective(problem),
        #  core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        (
            CVRAsObjective2(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
        # (
        #     CDFAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        (
            CDFAsObjective2(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, *args):
    if "dascmop" in problem_name:
        print(problem_name, args)
        problem = get_problem(problem_name, args[0])
    elif "modact" in problem_name:
        problem = get_problem(problem_name, args[0])
    else:
        print(problem_name)
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    print(
        f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
    )

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        # Return None in place of results to indicate failure
        return pf, [None] * len(results)

    # Prepare data for performance indicators, safely handling cases where result.F might be None or incorrectly shaped
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, "F") and result.F is not None:
            # i == 0 this NGSA without additional objective
            if i > 0:
                # if i == 1 or i == 3:
                # if True:
                prepared_data.append(result.F[:, :-2])
            else:
                # prepared_data.append(result.F[:, :-1])
                prepared_data.append(result.F)
        else:
            # Use None or an appropriate placeholder if result is invalid
            print(f'algo {i} not converge')
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            print(
                f"{metric}: " +
                " | ".join(f"{perf[i]}" for perf in performance_results)
            )
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, *results

# DASCMOP 1

In [8]:
pf, *results = test("dascmop1", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x10bcd9df0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0


KeyboardInterrupt: 

In [ ]:
pf, *results = test("dascmop2", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x119851df0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.350420 seconds
minimize_ executed in 28.104836 seconds
minimize_ executed in 28.199599 seconds
compare_on executed in 91.655464 seconds
Generational Distance (GD): (0.000455705515608654, 0.00038067415692797523) | (0.00048820798933302186, 0.0004016100085014275) | (0.00048820798933302186, 0.0004016100085014275)
Generational Distance Plus (GD+): (0.00041172834949077187, 0.00033422463738719266) | (0.00045104797067912035, 0.0003629294294660024) | (0.00045104797067912035, 0.0003629294294660024)
Inverted Generational Distance (IGD): (0.35792971763126563, 0.3577666491602637) | (0.3559409871699371, 0.3560573263230357) | (0.3559409871699371, 0.3560573263230357)
Inverted Generational Distance Plus (IGD+): (0.21068844909148923, 0.21047349582889344) | (0.2096061545729419, 0.20951445465448482) | (0.2096061545729419

(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.6836140e-01],
        [2.0020020e-03, 9.5525626e-01],
        ...,
        [9.9799800e-01, 1.0015025e-03],
        [9.9899900e-01, 5.0062581e-04],
        [1.0000000e+00, 0.0000000e+00]]),
 array([[0.34262427, 0.41976703],
        [0.30904496, 0.44835309],
        [0.34066315, 0.42099213],
        [0.33492721, 0.4229532 ],
        [0.34169737, 0.42034849],
        [0.30949996, 0.44797809],
        [0.33655678, 0.42208996],
        [0.30999911, 0.44602003],
        [0.33328419, 0.42361455],
        [0.33777523, 0.42150171],
        [0.33608478, 0.42281102],
        [0.33873554, 0.42107223],
        [0.31067039, 0.44373917],
        [0.31044007, 0.4447851 ],
        [0.31131722, 0.443455  ],
        [0.33825691, 0.42119168],
        [0.31163199, 0.44265719],
        [0.31254626, 0.44127874],
        [0.31209592, 0.44206061],
        [0.33278617, 0.42400498],
        [0.33244795, 0.42430354],
        [0.33307162, 0.42393717

In [ ]:
pf, *results = test("dascmop3", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x12829e130>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.475640 seconds
minimize_ executed in 29.820181 seconds
minimize_ executed in 30.925386 seconds
compare_on executed in 96.222199 seconds
Generational Distance (GD): (0.10921526080280093, 0.1882034356688524) | (0.203987802406345, 0.2055480429751696) | (0.203987802406345, 0.2055480429751696)
Generational Distance Plus (GD+): (0.10284326967523544, 0.18748348363811862) | (0.18898148692804578, 0.19032676585264735) | (0.18898148692804578, 0.19032676585264735)
Inverted Generational Distance (IGD): (0.46358405768598987, 0.4929121545651579) | (0.5158498697520715, 0.5145902249274354) | (0.5158498697520715, 0.5145902249274354)
Inverted Generational Distance Plus (IGD+): (0.3281555007458032, 0.39963056534361774) | (0.431033100506745, 0.4301682199529556) | (0.431033100506745, 0.4301682199529556)


(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.7622292e-01],
        [2.0020020e-03, 9.7097736e-01],
        [3.0030030e-03, 9.6877712e-01],
        [4.0040040e-03, 9.6814945e-01],
        [1.5015015e-01, 9.6522640e-01],
        [1.5115115e-01, 9.5832128e-01],
        [1.5215215e-01, 9.5133460e-01],
        [1.5315315e-01, 9.4426774e-01],
        [1.5415415e-01, 9.3712209e-01],
        [1.5515516e-01, 9.2989905e-01],
        [1.5615616e-01, 9.2260006e-01],
        [1.5715716e-01, 9.1522656e-01],
        [1.5815816e-01, 9.0778004e-01],
        [1.9219219e-01, 6.2277146e-01],
        [1.9319319e-01, 6.1382126e-01],
        [1.9419419e-01, 6.0486081e-01],
        [1.9519520e-01, 5.9589203e-01],
        [1.9619620e-01, 5.8691683e-01],
        [1.9719720e-01, 5.7793714e-01],
        [1.9819820e-01, 5.6895487e-01],
        [1.9919920e-01, 5.5997193e-01],
        [2.0020020e-01, 5.5413500e-01],
        [3.9239239e-01, 4.3319559e-01],
        [3.9339339e-01, 4.2458428e-01],


In [ ]:
pf, *results = test("dascmop4", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x128619f70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 37.240217 seconds
minimize_ executed in 31.254633 seconds
minimize_ executed in 31.404174 seconds
compare_on executed in 99.899605 seconds
Generational Distance (GD): (0.0016016104395435095, 0.0015559532687730267) | (0.028254980371941248, 0.02900179616615087) | (0.028254980371941248, 0.02900179616615087)
Generational Distance Plus (GD+): (0.001600815868752598, 0.0015559532687730267) | (0.02825147077351776, 0.02900179616615087) | (0.02825147077351776, 0.02900179616615087)
Inverted Generational Distance (IGD): (0.47371202043421967, 0.43107283057998463) | (0.4824856380727934, 0.4473054739922206) | (0.4824856380727934, 0.4473054739922206)
Inverted Generational Distance Plus (IGD+): (0.3370957052878546, 0.31090932860221754) | (0.35803195009258476, 0.33911722694215213) | (0.35803195009258476, 0.339117226942152

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[1.0005609 , 0.00152263],
        [0.00171777, 1.00103697],
        [0.99298168, 0.01653342],
        [0.95868008, 0.08317311],
        [0.89437063, 0.20533954],
        [0.85897811, 0.26491579],
        [0.65899862, 0.56828198],
        [0.75948365, 0.42659315],
        [0.79324766, 0.37395127],
        [0.69421288, 0.521269  ],
        [0.55851009, 0.69025879],
        [0.59292492, 0.65113057],
        [0.4930885 , 0.75905248],
        [0.45809212, 0.79199495],
        [0.1536844 , 0.97872596],
        [0.35735848, 0.87337598],
        [0.3930763 , 0.8478371 ],
        [0.19632753, 0.96398092],
        [0.25710008, 0.93567514],
        [0.29306309, 0.91616131],
        [0.09470261, 0.9919104 ],
        [0.05845098, 0.99770042],
        [0.00821544, 1.00075849],

In [ ]:
pf, *results = test("dascmop5", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x128f52220>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 34.788658 seconds
minimize_ executed in 32.532431 seconds
minimize_ executed in 34.378931 seconds
compare_on executed in 101.700708 seconds
Generational Distance (GD): (0.01794802439331465, 0.018097239477250313) | (0.008046942965075977, 0.0066393385173847495) | (0.008046942965075977, 0.0066393385173847495)
Generational Distance Plus (GD+): (0.017941298936362256, 0.018097239477250313) | (0.008044265747881587, 0.0066393385173847495) | (0.008044265747881587, 0.0066393385173847495)
Inverted Generational Distance (IGD): (0.4499135656657036, 0.40140491380533555) | (0.4665729224555891, 0.43124505166694804) | (0.4665729224555891, 0.43124505166694804)
Inverted Generational Distance Plus (IGD+): (0.32366980219645836, 0.2900050627916686) | (0.3317850605889466, 0.30577809697907704) | (0.3317850605889466, 0.305778096

(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.6836140e-01],
        [2.0020020e-03, 9.5525626e-01],
        ...,
        [9.9799800e-01, 1.0015025e-03],
        [9.9899900e-01, 5.0062581e-04],
        [1.0000000e+00, 0.0000000e+00]]),
 array([[0.40815968, 0.38569983],
        [0.54875304, 0.28013748],
        [0.04743374, 0.82814528],
        [0.72991889, 0.16621969],
        [0.92655623, 0.05764893],
        [0.40843268, 0.38536553],
        [0.64258402, 0.22063033],
        [0.46315287, 0.34228058],
        [0.42122164, 0.37596902],
        [0.71233207, 0.17761606],
        [0.44704442, 0.35341224],
        [0.60614308, 0.24303442],
        [0.62747706, 0.22939106],
        [0.5197814 , 0.30065273],
        [0.95632666, 0.0419017 ],
        [0.70808578, 0.18173873],
        [0.66972794, 0.20566908],
        [0.14654449, 0.64481826],
        [0.54062308, 0.28590865],
        [0.25826197, 0.51835367],
        [0.24191613, 0.5374997 ],
        [0.86027966, 0.09255996

In [ ]:
pf, *results = test("dascmop6", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x12876eeb0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.527026 seconds
minimize_ executed in 31.581716 seconds
minimize_ executed in 32.264916 seconds
compare_on executed in 99.374321 seconds
Generational Distance (GD): (0.0025979362180255966, 0.0023816302059004317) | (0.0022095623318156036, 0.0020041530815166717) | (0.0022095623318156036, 0.0020041530815166717)
Generational Distance Plus (GD+): (0.0017139409037969543, 0.0017020752421443297) | (0.0010433702833314105, 0.0009629750458598068) | (0.0010433702833314105, 0.0009629750458598068)
Inverted Generational Distance (IGD): (0.5023717140124812, 0.4771033101959869) | (0.5319108259991261, 0.5048070085628089) | (0.5319108259991261, 0.5048070085628089)
Inverted Generational Distance Plus (IGD+): (0.3293552704784667, 0.2676817780707894) | (0.36447810151808513, 0.3638704081808506) | (0.36447810151808513, 0.3638

(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.7622292e-01],
        [2.0020020e-03, 9.7097736e-01],
        [3.0030030e-03, 9.6877712e-01],
        [4.0040040e-03, 9.6814945e-01],
        [1.5015015e-01, 9.6522640e-01],
        [1.5115115e-01, 9.5832128e-01],
        [1.5215215e-01, 9.5133460e-01],
        [1.5315315e-01, 9.4426774e-01],
        [1.5415415e-01, 9.3712209e-01],
        [1.5515516e-01, 9.2989905e-01],
        [1.5615616e-01, 9.2260006e-01],
        [1.5715716e-01, 9.1522656e-01],
        [1.5815816e-01, 9.0778004e-01],
        [1.9219219e-01, 6.2277146e-01],
        [1.9319319e-01, 6.1382126e-01],
        [1.9419419e-01, 6.0486081e-01],
        [1.9519520e-01, 5.9589203e-01],
        [1.9619620e-01, 5.8691683e-01],
        [1.9719720e-01, 5.7793714e-01],
        [1.9819820e-01, 5.6895487e-01],
        [1.9919920e-01, 5.5997193e-01],
        [2.0020020e-01, 5.5413500e-01],
        [3.9239239e-01, 4.3319559e-01],
        [3.9339339e-01, 4.2458428e-01],


In [ ]:
pf, *results = test("dascmop7", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x128eb0070>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 36.175920 seconds
minimize_ executed in 31.274945 seconds
minimize_ executed in 29.800746 seconds
compare_on executed in 97.255487 seconds
Generational Distance (GD): (0.11640499346956724, 0.1160256951343667) | (0.06277571358397076, 0.059829069512292915) | (0.06712386861859995, 0.0654437868589143)
Generational Distance Plus (GD+): (0.11640499346956724, 0.1160256951343667) | (0.06277571358397076, 0.059829069512292915) | (0.06712386861859995, 0.0654437868589143)
Inverted Generational Distance (IGD): (0.6175733445051123, 0.5921912958530791) | (0.6211687145270088, 0.5872020620843137) | (0.5993839338962353, 0.5633796944096505)
Inverted Generational Distance Plus (IGD+): (0.4967247772465468, 0.4733415236858973) | (0.47338120187862764, 0.44372119905123897) | (0.45747075700467577, 0.4312384515619048)


(array([[0.        , 0.        , 1.        ],
        [0.        , 0.01010101, 0.98989899],
        [0.        , 0.02020202, 0.97979798],
        ...,
        [0.97979798, 0.        , 0.02020202],
        [0.98989899, 0.        , 0.01010101],
        [1.        , 0.        , 0.        ]]),
 array([[0.3768333 , 0.05327002, 0.72460776],
        [1.05651227, 0.07235276, 0.0663814 ],
        [0.75684087, 0.36141372, 0.05999995],
        ...,
        [0.43495521, 0.4150999 , 0.3604062 ],
        [0.50438965, 0.32943414, 0.36124811],
        [0.1849722 , 0.31869412, 0.69747689]]),
 array([[1.02869734, 0.03321032, 0.03585418, 0.        , 0.        ],
        [0.9231175 , 0.03211366, 0.13904682, 0.        , 0.        ],
        [0.03860072, 1.02503669, 0.0392628 , 0.        , 0.        ],
        ...,
        [0.11759373, 0.66824888, 0.31270074, 0.        , 0.        ],
        [0.28967373, 0.38211681, 0.42699027, 0.        , 0.        ],
        [0.23891627, 0.15536292, 0.70256732, 0.        

In [ ]:
pf, *results = test("dascmop8", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x12883b1c0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 37.619006 seconds
minimize_ executed in 33.966665 seconds
minimize_ executed in 32.597747 seconds
compare_on executed in 104.183979 seconds
Generational Distance (GD): (0.11135486412025791, 0.11230583181583559) | (0.14653205002824682, 0.1489168830391281) | (0.10423159772912743, 0.1043489321630153)
Generational Distance Plus (GD+): (0.11130249639716695, 0.11230583181583559) | (0.14651027740094857, 0.1489168830391281) | (0.10417121482477137, 0.1043489321630153)
Inverted Generational Distance (IGD): (0.6936903156424548, 0.6717486929346507) | (0.7000512338045143, 0.6758694471225328) | (0.6805051370188228, 0.6560582549069252)
Inverted Generational Distance Plus (IGD+): (0.5660007843845887, 0.5766117561148634) | (0.5906564402756227, 0.5971980162795236) | (0.5561154408238856, 0.5588687126936951)


(array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [9.9987413e-01, 1.5865964e-02, 0.0000000e+00],
        [9.9949654e-01, 3.1727933e-02, 0.0000000e+00],
        ...,
        [1.9427756e-18, 6.1201512e-17, 1.0000000e+00],
        [9.7151009e-19, 6.1224633e-17, 1.0000000e+00],
        [3.7493995e-33, 6.1232340e-17, 1.0000000e+00]]),
 array([[0.23837334, 1.04794653, 0.06589077],
        [0.06682766, 0.28819727, 1.04027239],
        [0.07383097, 1.06743419, 0.06763756],
        ...,
        [1.02374923, 0.24369669, 0.31115414],
        [0.19591244, 0.50458008, 0.96918322],
        [0.44081441, 0.25215935, 0.97660747]]),
 array([[0.11292729, 1.0865515 , 0.08821797, 0.        , 0.        ],
        [0.08801656, 0.9415913 , 0.60037005, 0.        , 0.        ],
        [0.46799734, 0.08205373, 0.99799531, 0.        , 0.        ],
        ...,
        [0.74464821, 0.1750842 , 0.83967902, 0.        , 0.        ],
        [0.97312019, 0.54262413, 0.23966763, 0.        , 0.        ],
 

In [ ]:
pf, *results = test("dascmop9", 500, 1)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (1,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x128f7fd30>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 33.188122 seconds
minimize_ executed in 30.652350 seconds
minimize_ executed in 32.039958 seconds
compare_on executed in 95.881003 seconds
Generational Distance (GD): (0.009127754919616739, 0.0053566976864764075) | (0.00541522663023606, 0.004786921006780843) | (0.005282245022372775, 0.004983416912348282)
Generational Distance Plus (GD+): (0.00802414851849921, 0.004081439253192322) | (0.003364295277616379, 0.0021909437367563234) | (0.0030501255937501136, 0.0021933222051744583)
Inverted Generational Distance (IGD): (0.6417809952442809, 0.6364609175376088) | (0.6654176996631768, 0.6534494150664001) | (0.6694382377233064, 0.6581693816371768)
Inverted Generational Distance Plus (IGD+): (0.4180434721443665, 0.4037477219221153) | (0.39278844518976724, 0.3897395479816284) | (0.3912318502992327, 0.3898756357098398

(array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [9.9987413e-01, 1.5865964e-02, 0.0000000e+00],
        [9.9949654e-01, 3.1727933e-02, 0.0000000e+00],
        ...,
        [1.9427756e-18, 6.1201512e-17, 1.0000000e+00],
        [9.7151009e-19, 6.1224633e-17, 1.0000000e+00],
        [3.7493995e-33, 6.1232340e-17, 1.0000000e+00]]),
 array([[9.81759332e-04, 1.15535622e-03, 1.00059196e+00],
        [4.02198374e-04, 5.82884228e-01, 8.13182473e-01],
        [2.15846122e-01, 1.21009373e+00, 3.04989581e-01],
        ...,
        [5.98664433e-02, 1.19234975e-01, 9.92212449e-01],
        [1.77657829e-02, 5.32626588e-01, 8.47777751e-01],
        [2.30477538e-03, 6.48601178e-01, 7.64517925e-01]]),
 array([[6.24167464e-02, 7.07744415e-01, 8.25558229e-01, 0.00000000e+00,
         0.00000000e+00],
        [4.29388758e-04, 4.41467749e-04, 1.00042191e+00, 0.00000000e+00,
         0.00000000e+00],
        [6.16087383e-02, 7.33547579e-01, 8.01537443e-01, 0.00000000e+00,
         0.00000000e+

# DASCMOP 2

In [ ]:
pf, *results = test("dascmop1", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x128543880>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.119393 seconds
minimize_ executed in 28.583099 seconds
minimize_ executed in 27.771090 seconds
compare_on executed in 88.474270 seconds
Generational Distance (GD): (0.00038500201317177716, 0.0003816569289176526) | (0.00036549047166715894, 0.0003450623291668659) | (0.00036549047166715894, 0.0003450623291668659)
Generational Distance Plus (GD+): (0.0003244569798107617, 0.00030528663206891293) | (0.00032128643111026907, 0.0002945203049435352) | (0.00032128643111026907, 0.0002945203049435352)
Inverted Generational Distance (IGD): (0.4114048787289414, 0.40975438128181196) | (0.3987063473858393, 0.39647171267018944) | (0.3987063473858393, 0.39647171267018944)
Inverted Generational Distance Plus (IGD+): (0.3073252401625841, 0.30709516899530226) | (0.30392024903061843, 0.3034889258133006) | (0.303920249030618

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.81614774, 1.40012469],
        [0.68397528, 1.46663167],
        [0.8167863 , 1.3996898 ],
        [0.7682085 , 1.4281055 ],
        [0.75983413, 1.43251357],
        [0.7356073 , 1.44474936],
        [0.8168629 , 1.399611  ],
        [0.76217311, 1.43129115],
        [0.76768213, 1.42840581],
        [0.77665813, 1.42356831],
        [0.76283942, 1.43094073],
        [0.80153402, 1.40913324],
        [0.8246235 , 1.39474484],
        [0.77330196, 1.42561078],
        [0.79612445, 1.41235965],
        [0.76869351, 1.42802922],
        [0.76166547, 1.43154575],
        [0.81263031, 1.40229905],
        [0.81757891, 1.39915893],
        [0.77039415, 1.42690863],
        [0.71336437, 1.45470358],
        [0.8498206 , 1.37768606],
        [0.79698747, 1.41195289],
        [0.79494111, 1.

In [ ]:
pf, *results = test("dascmop2", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x12873c3d0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.058655 seconds
minimize_ executed in 31.797329 seconds
minimize_ executed in 31.548660 seconds
compare_on executed in 96.405339 seconds
Generational Distance (GD): (0.00042514112874699736, 0.0004038941257544453) | (0.0004350550846594842, 0.00041825630012223776) | (0.0004350550846594842, 0.00041825630012223776)
Generational Distance Plus (GD+): (0.00038138006414447663, 0.0003431931337328775) | (0.00038654770709923485, 0.00035976053419228826) | (0.00038654770709923485, 0.00035976053419228826)
Inverted Generational Distance (IGD): (0.3617252589012029, 0.3596399728949149) | (0.3498895288259838, 0.3483583335174343) | (0.3498895288259838, 0.3483583335174343)
Inverted Generational Distance Plus (IGD+): (0.21384016534323522, 0.21146470848332982) | (0.20755745965746866, 0.2054400262293844) | (0.207557459657468

(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.82402452, 0.93082728],
        [0.88948919, 0.87618055],
        [0.89084821, 0.87498174],
        [0.82507648, 0.92987207],
        [0.80762037, 0.94549439],
        [0.87204124, 0.89040445],
        [0.81030365, 0.94296875],
        [0.779152  , 0.97167791],
        [0.88297614, 0.88122942],
        [0.80610384, 0.94677722],
        [0.74895794, 1.00163674],
        [0.75880415, 0.99130963],
        [0.82935606, 0.92622677],
        [0.75629436, 0.99395181],
        [0.77592193, 0.97479705],
        [0.81300075, 0.94083874],
        [0.82233828, 0.93249779],
        [0.7897201 , 0.96191319],
        [0.81021038, 0.9435352 ],
        [0.75190289, 0.9983849 ],
        [0.8046021 , 0.94809565],
        [0.86325648, 0.89746578],
        [0.80695086, 0.94609876],

In [ ]:
pf, *results = test("dascmop3", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x128eb0070>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.064709 seconds
minimize_ executed in 28.984047 seconds
minimize_ executed in 29.747300 seconds
compare_on executed in 93.796621 seconds
Generational Distance (GD): (0.0677688142234022, 0.03727952820774038) | (0.05008679286748155, 0.0191412633555389) | (0.05008679286748155, 0.0191412633555389)
Generational Distance Plus (GD+): (0.06682829650530585, 0.0370036279407564) | (0.049050825089261396, 0.019010986427550947) | (0.049050825089261396, 0.019010986427550947)
Inverted Generational Distance (IGD): (0.38392453773579394, 0.3762012009020987) | (0.3757966948063998, 0.3732291365514697) | (0.3757966948063998, 0.3732291365514697)
Inverted Generational Distance Plus (IGD+): (0.2730142109449691, 0.2683926747486256) | (0.25830670597948246, 0.2595153777672168) | (0.25830670597948246, 0.2595153777672168)


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [ ]:
pf, *results = test("dascmop4", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x128eb0100>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 31.622230 seconds
minimize_ executed in 31.360157 seconds
minimize_ executed in 31.015285 seconds
compare_on executed in 93.998396 seconds
Generational Distance (GD): (0.0008255508868399009, 0.0007091380557383608) | (0.0006679603898944544, 0.0006021506118920512) | (0.0006679603898944544, 0.0006021506118920512)
Generational Distance Plus (GD+): (0.0007883171229702212, 0.000640354407582964) | (0.0006224354603359389, 0.0005352324086360177) | (0.0006224354603359389, 0.0005352324086360177)
Inverted Generational Distance (IGD): (0.48251669405534914, 0.43470424320117934) | (0.4884671814694409, 0.44147004707165804) | (0.4884671814694409, 0.44147004707165804)
Inverted Generational Distance Plus (IGD+): (0.3411768064818436, 0.3178990101584507) | (0.3432225308865899, 0.3196626800484347) | (0.3432225308865899, 0.319

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.50104034, 1.50035461],
        [1.5003994 , 0.5006529 ],
        [0.56507546, 1.49819801],
        [0.54445795, 1.49821525],
        [0.58598068, 1.49441902],
        [0.5112511 , 1.50013604],
        [1.41929259, 0.65825096],
        [1.41303162, 0.66857341],
        [0.52179871, 1.49983002],
        [0.98471448, 1.26569135],
        [1.44134012, 0.6142603 ],
        [1.06442805, 1.18180523],
        [0.59613411, 1.49128615],
        [0.60646109, 1.4891834 ],
        [0.5316739 , 1.49933674],
        [0.99099351, 1.25947705],
        [1.43773614, 0.62169196],
        [1.24484314, 0.94542131],
        [1.46988269, 0.56127025],
        [1.39470327, 0.70038824],
        [1.4461942 , 0.60507452],
        [1.39794168, 0.69379516],
        [1.34186546, 0.79217533],
        [1.24150448, 0.

In [ ]:
pf, *results = test("dascmop5", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x1319701f0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.925367 seconds
minimize_ executed in 25.054403 seconds
minimize_ executed in 23.947781 seconds
compare_on executed in 81.928238 seconds
Generational Distance (GD): (0.0008700235958134116, 0.0006982162310429888) | (0.0007825269874324541, 0.0006416397365206989) | (0.0007825269874324541, 0.0006416397365206989)
Generational Distance Plus (GD+): (0.0007118576828853225, 0.0006313345268056663) | (0.0006988214128485453, 0.0005942577846104818) | (0.0006988214128485453, 0.0005942577846104818)
Inverted Generational Distance (IGD): (0.4602777705556622, 0.4232706987792889) | (0.44742731163264476, 0.40851103085526513) | (0.44742731163264476, 0.40851103085526513)
Inverted Generational Distance Plus (IGD+): (0.32052073155698846, 0.2871537231364245) | (0.30925240927620773, 0.27352425329274843) | (0.30925240927620773, 

(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.50041095, 1.48477054],
        [1.50000839, 0.50176618],
        [1.2305118 , 0.64541845],
        [0.73940148, 1.011705  ],
        [1.23757861, 0.64217842],
        [1.29554445, 0.60856641],
        [0.73541473, 1.01584016],
        [0.52631756, 1.33803535],
        [1.2883434 , 0.61229813],
        [0.76189676, 0.98826814],
        [0.63983487, 1.12657864],
        [0.94931187, 0.82985196],
        [0.50217717, 1.45773246],
        [1.27424137, 0.62056189],
        [0.50388739, 1.439828  ],
        [0.79154155, 0.96053918],
        [0.50255327, 1.44961746],
        [1.16899461, 0.68329997],
        [0.50799611, 1.41080342],
        [1.3733051 , 0.56576607],
        [0.98214532, 0.80696284],
        [0.50164111, 1.46135401],
        [0.92969385, 0.84593663],

In [ ]:
pf, *results = test("dascmop5", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x128e8fc70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 36.096102 seconds
minimize_ executed in 32.473433 seconds
minimize_ executed in 30.733604 seconds
compare_on executed in 99.303829 seconds
Generational Distance (GD): (0.18568481474301352, 0.18798807793681016) | (0.18568481474301352, 0.18798807793681016) | (0.18568481474301352, 0.18798807793681016)
Generational Distance Plus (GD+): (0.1856833154826525, 0.18798807793681016) | (0.1856833154826525, 0.18798807793681016) | (0.1856833154826525, 0.18798807793681016)
Inverted Generational Distance (IGD): (0.5028990239771907, 0.46258946576222804) | (0.5028990239771907, 0.46258946576222804) | (0.5028990239771907, 0.46258946576222804)
Inverted Generational Distance Plus (IGD+): (0.44557540180178173, 0.4021815394102295) | (0.44557540180178173, 0.4021815394102295) | (0.44557540180178173, 0.4021815394102295)


(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.6836140e-01],
        [2.0020020e-03, 9.5525626e-01],
        ...,
        [9.9799800e-01, 1.0015025e-03],
        [9.9899900e-01, 5.0062581e-04],
        [1.0000000e+00, 0.0000000e+00]]),
 array([[0.5145535 , 0.52242151],
        [0.34948748, 0.67943601],
        [1.0534047 , 0.18260183],
        [0.48865687, 0.53455247],
        [0.37528487, 0.65666597],
        [0.579401  , 0.47127001],
        [0.5160611 , 0.51966055],
        [1.03592295, 0.19621061],
        [0.62467148, 0.43641909],
        [0.50616003, 0.52403076],
        [0.82935029, 0.31264783],
        [0.30069613, 0.73540509],
        [1.11804637, 0.14622409],
        [0.36097242, 0.67098065],
        [0.28239141, 0.75705681],
        [0.38149395, 0.63440805],
        [0.98012094, 0.22299115],
        [0.17030094, 0.95653455],
        [0.34856878, 0.68097129],
        [0.56317922, 0.48719597],
        [0.28907611, 0.74915026],
        [0.47271188, 0.55436417

In [ ]:
pf, *results = test("dascmop6", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x12870d490>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.494786 seconds
minimize_ executed in 29.952255 seconds
minimize_ executed in 31.159657 seconds
compare_on executed in 96.607276 seconds
Generational Distance (GD): (0.0022268347574008595, 0.002261072127406331) | (0.002060987273704893, 0.0019520253826348687) | (0.002060987273704893, 0.0019520253826348687)
Generational Distance Plus (GD+): (0.0007856363974467592, 0.0007259300782099798) | (0.0005661713719596782, 0.0005718661305659368) | (0.0005661713719596782, 0.0005718661305659368)
Inverted Generational Distance (IGD): (0.4763091853514809, 0.43168314502006033) | (0.44099873686578195, 0.42304910263911655) | (0.44099873686578195, 0.42304910263911655)
Inverted Generational Distance Plus (IGD+): (0.3283116479734437, 0.2870298800129495) | (0.29670953103034226, 0.2784734533648671) | (0.29670953103034226, 0.27

(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [ ]:
pf, *results = test("dascmop7", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x131986dc0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 34.613086 seconds
minimize_ executed in 32.100391 seconds
minimize_ executed in 31.858673 seconds
compare_on executed in 98.573094 seconds
Generational Distance (GD): (0.010422812267963977, 0.008003829313203628) | (0.011281935199198664, 0.008649901172166428) | (0.011281935199198664, 0.008649901172166428)
Generational Distance Plus (GD+): (0.010230877275495973, 0.007923050015075934) | (0.011081630793685269, 0.00854038257306398) | (0.011081630793685269, 0.00854038257306398)
Inverted Generational Distance (IGD): (0.5985914539724383, 0.5744975769182862) | (0.6050604018302904, 0.5755562237281826) | (0.6050604018302904, 0.5755562237281826)
Inverted Generational Distance Plus (IGD+): (0.42626376413830264, 0.4006339625396678) | (0.4315622087598899, 0.40321175543811005) | (0.4315622087598899, 0.40321175543811005)


(array([[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.51010101, 1.489899  ],
        [0.5       , 0.52020202, 1.479798  ],
        ...,
        [1.479798  , 0.5       , 0.52020202],
        [1.489899  , 0.5       , 0.51010101],
        [1.5       , 0.5       , 0.5       ]]),
 array([[0.50068745, 0.52923104, 1.47116065],
        [1.4943057 , 0.50502913, 0.50472995],
        [0.50174624, 0.50107363, 1.50036198],
        ...,
        [0.74796698, 0.68798962, 1.06505953],
        [0.64658716, 1.34906172, 0.51205622],
        [0.67106014, 0.71909457, 1.15996755]]),
 array([[0.50257259, 0.50268424, 1.49947902, 0.        , 0.        ],
        [1.49844747, 0.50174638, 0.50375006, 0.        , 0.        ],
        [0.79988642, 1.2000939 , 0.50050855, 0.        , 0.        ],
        ...,
        [0.79524785, 0.73310719, 0.97653478, 0.        , 0.        ],
        [1.46746463, 0.5166148 , 0.5249466 , 0.        , 0.        ],
        [0.53852445, 1.02764966, 0.93884538, 0.        

In [ ]:
pf, *results = test("dascmop8", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x13187d7f0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 35.184382 seconds
minimize_ executed in 30.647381 seconds
minimize_ executed in 30.802891 seconds
compare_on executed in 96.635184 seconds
Generational Distance (GD): (0.010017439989109003, 0.008556286206999768) | (0.009666883840864029, 0.008031809925024625) | (0.009666883840864029, 0.008031809925024625)
Generational Distance Plus (GD+): (0.009131003211866176, 0.007362974440804462) | (0.008716725086482222, 0.006913831868114763) | (0.008716725086482222, 0.006913831868114763)
Inverted Generational Distance (IGD): (0.673841071398447, 0.6444623639040222) | (0.6729529057523226, 0.6486903607033301) | (0.6729529057523226, 0.6486903607033301)
Inverted Generational Distance Plus (IGD+): (0.4892366379601835, 0.49802665530522594) | (0.48955159846536456, 0.4977666534552654) | (0.48955159846536456, 0.4977666534552654)

(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.50339121, 1.50119824, 0.51060479],
        [0.50037415, 0.50047272, 1.50009232],
        [0.59730586, 1.49553421, 0.50042149],
        ...,
        [0.99847809, 1.03758579, 1.19746137],
        [1.28709004, 0.72684235, 1.1160607 ],
        [0.74862284, 0.92484684, 1.37955226]]),
 array([[0.50222554, 1.50081888, 0.50236385, 0.        , 0.        ],
        [1.25629171, 1.15521041, 0.50064271, 0.        , 0.        ],
        [0.50054977, 1.40344373, 0.92918075, 0.        , 0.        ],
        ...,
        [0.75263763, 1.2339123 , 1.14150404, 0.        , 0.        ],
        [0.88452528, 0.6475906 , 1.41323946, 0.        , 0.        ],
        [1.06174917, 0.64995098, 1.31748433, 0.        

In [ ]:
pf, *results = test("dascmop9", 500, 2)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (2,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x128e8fa60>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 34.548425 seconds
minimize_ executed in 30.461210 seconds
minimize_ executed in 29.785245 seconds
compare_on executed in 94.795433 seconds
Generational Distance (GD): (0.006064014096744169, 0.005630609224927261) | (0.006515265150817819, 0.005972971884400345) | (0.006515265150817819, 0.005972971884400345)
Generational Distance Plus (GD+): (0.004680285789793981, 0.00401728274544389) | (0.005022914827766705, 0.004233199636489382) | (0.005022914827766705, 0.004233199636489382)
Inverted Generational Distance (IGD): (0.613644225960163, 0.6051677767517258) | (0.6088978146934976, 0.5883245137124974) | (0.6088978146934976, 0.5883245137124974)
Inverted Generational Distance Plus (IGD+): (0.4355552323132211, 0.42802569685033165) | (0.4342861502928443, 0.42325031593149415) | (0.4342861502928443, 0.42325031593149415)


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.96334322, 1.22664638, 1.05934046],
        [0.50713644, 1.45512401, 0.82462496],
        [0.50021697, 1.44373306, 0.83075553],
        ...,
        [0.57844698, 1.18025194, 1.23054145],
        [0.64955846, 0.91647165, 1.39698676],
        [0.52003743, 0.97023529, 1.39663403]]),
 array([[0.52402859, 1.49284219, 0.72095504, 0.        , 0.        ],
        [1.11838038, 0.98149013, 1.14939379, 0.        , 0.        ],
        [0.50244038, 0.50237506, 1.50025051, 0.        , 0.        ],
        ...,
        [0.62703543, 1.15491088, 1.24576763, 0.        , 0.        ],
        [0.66261381, 0.84361001, 1.43343912, 0.        , 0.        ],
        [0.63824319, 1.05812177, 1.32240582, 0.        

# DASCMOP 3

In [ ]:
pf, *results = test("dascmop1", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x11181c7f0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.361236 seconds
minimize_ executed in 30.718421 seconds
minimize_ executed in 40.435300 seconds
compare_on executed in 104.515661 seconds
Generational Distance (GD): (0.0004141529816857805, 0.00034159388917473743) | (0.0004141529816857805, 0.00034159388917473743) | (0.0004141529816857805, 0.00034159388917473743)
Generational Distance Plus (GD+): (0.0003610739057811906, 0.0002728951869730434) | (0.0003610739057811906, 0.0002728951869730434) | (0.0003610739057811906, 0.0002728951869730434)
Inverted Generational Distance (IGD): (0.4039831500451796, 0.4039270733548538) | (0.4039831500451796, 0.4039270733548538) | (0.4039831500451796, 0.4039270733548538)
Inverted Generational Distance Plus (IGD+): (0.30562422326107724, 0.30550606203654007) | (0.30562422326107724, 0.30550606203654007) | (0.30562422326107724,

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[0.29815177, 0.91771136],
        [0.32557105, 0.89692135],
        [0.29855751, 0.91477699],
        [0.32056208, 0.898467  ],
        [0.32410492, 0.89739781],
        [0.32234111, 0.89784193],
        [0.29947486, 0.91287181],
        [0.31883699, 0.89914368],
        [0.29946799, 0.91364581],
        [0.29929751, 0.91457694],
        [0.2997758 , 0.91183452],
        [0.30031478, 0.91124776],
        [0.32520263, 0.89714658],
        [0.32468885, 0.89737455],
        [0.32129829, 0.8980523 ],
        [0.29997091, 0.91181592],
        [0.32091643, 0.89842284],
        [0.30118044, 0.91018428],
        [0.31760616, 0.89953496],
        [0.30075116, 0.91058725],
        [0.30094859, 0.91026954],
        [0.30139249, 0.90978073],
        [0.30050754, 0.91089755],

In [ ]:
pf, *results = test("dascmop2", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x1318ff280>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 34.566403 seconds
minimize_ executed in 30.154141 seconds
minimize_ executed in 29.766414 seconds
compare_on executed in 94.487709 seconds
Generational Distance (GD): (0.0004780541252617823, 0.00039880484104647004) | (0.0004780541252617823, 0.00039880484104647004) | (0.0004780541252617823, 0.00039880484104647004)
Generational Distance Plus (GD+): (0.0004303049719668536, 0.0003378709456926532) | (0.0004303049719668536, 0.0003378709456926532) | (0.0004303049719668536, 0.0003378709456926532)
Inverted Generational Distance (IGD): (0.3528665013672096, 0.3525326191661835) | (0.3528665013672096, 0.3525326191661835) | (0.3528665013672096, 0.3525326191661835)
Inverted Generational Distance Plus (IGD+): (0.2078601991289519, 0.2075289984594192) | (0.2078601991289519, 0.2075289984594192) | (0.2078601991289519, 0.207

(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.6836140e-01],
        [2.0020020e-03, 9.5525626e-01],
        ...,
        [9.9799800e-01, 1.0015025e-03],
        [9.9899900e-01, 5.0062581e-04],
        [1.0000000e+00, 0.0000000e+00]]),
 array([[0.35822668, 0.40760017],
        [0.32276582, 0.44301499],
        [0.32491813, 0.43523106],
        [0.35342579, 0.40835098],
        [0.32555359, 0.43248223],
        [0.3249351 , 0.43477839],
        [0.32631224, 0.43160361],
        [0.35056243, 0.40942371],
        [0.32735916, 0.42988805],
        [0.32671593, 0.43079737],
        [0.35155105, 0.40907546],
        [0.35217209, 0.40885154],
        [0.32780667, 0.42917286],
        [0.32713209, 0.43014526],
        [0.34946699, 0.41017533],
        [0.34988906, 0.40987711],
        [0.35283684, 0.40850551],
        [0.32672519, 0.43040766],
        [0.32795335, 0.42835623],
        [0.35280225, 0.40884278],
        [0.32785352, 0.4291087 ],
        [0.32876574, 0.4278599 

In [ ]:
pf, *results = test("dascmop3", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x1286b0460>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.965319 seconds
minimize_ executed in 31.281820 seconds
minimize_ executed in 31.721714 seconds
compare_on executed in 96.969532 seconds
Generational Distance (GD): (0.0021889516533603773, 0.002154469865703801) | (0.0021889516533603773, 0.002154469865703801) | (0.0021889516533603773, 0.002154469865703801)
Generational Distance Plus (GD+): (0.0005881890451641955, 0.0005475816826643054) | (0.0005881890451641955, 0.0005475816826643054) | (0.0005881890451641955, 0.0005475816826643054)
Inverted Generational Distance (IGD): (0.37008912932315163, 0.36778658041083373) | (0.37008912932315163, 0.36778658041083373) | (0.37008912932315163, 0.36778658041083373)
Inverted Generational Distance Plus (IGD+): (0.20339091127664252, 0.201650632948346) | (0.20339091127664252, 0.201650632948346) | (0.20339091127664252, 0.20

(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.7622292e-01],
        [2.0020020e-03, 9.7097736e-01],
        [3.0030030e-03, 9.6877712e-01],
        [4.0040040e-03, 9.6814945e-01],
        [1.5015015e-01, 9.6522640e-01],
        [1.5115115e-01, 9.5832128e-01],
        [1.5215215e-01, 9.5133460e-01],
        [1.5315315e-01, 9.4426774e-01],
        [1.5415415e-01, 9.3712209e-01],
        [1.5515516e-01, 9.2989905e-01],
        [1.5615616e-01, 9.2260006e-01],
        [1.5715716e-01, 9.1522656e-01],
        [1.5815816e-01, 9.0778004e-01],
        [1.5915916e-01, 9.0026198e-01],
        [1.6016016e-01, 8.9267389e-01],
        [1.6116116e-01, 8.8501730e-01],
        [1.6216216e-01, 8.7729376e-01],
        [1.6316316e-01, 8.6950484e-01],
        [1.6416416e-01, 8.6165211e-01],
        [1.6516517e-01, 8.5373718e-01],
        [1.6616617e-01, 8.4576167e-01],
        [1.6716717e-01, 8.3772721e-01],
        [1.6816817e-01, 8.2963546e-01],
        [1.6916917e-01, 8.2148807e-01],


In [ ]:
pf, *results = test("dascmop4", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x12876eeb0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.398045 seconds
minimize_ executed in 32.528421 seconds
minimize_ executed in 32.377794 seconds
compare_on executed in 100.304748 seconds
Generational Distance (GD): (0.4191513481790067, 0.4425231344500867) | (0.4191513481790067, 0.4425231344500867) | (0.4191513481790067, 0.4425231344500867)
Generational Distance Plus (GD+): (0.4191513481790067, 0.4425231344500867) | (0.4191513481790067, 0.4425231344500867) | (0.4191513481790067, 0.4425231344500867)
Inverted Generational Distance (IGD): (0.7453682033929032, 0.7655055135961202) | (0.7453682033929032, 0.7655055135961202) | (0.7453682033929032, 0.7655055135961202)
Inverted Generational Distance Plus (IGD+): (0.7102312398260797, 0.7346108592369408) | (0.7102312398260797, 0.7346108592369408) | (0.7102312398260797, 0.7346108592369408)


(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[1.25336109, 0.48646943],
        [1.23743577, 0.51225158],
        [1.26456951, 0.46600654],
        [1.27622336, 0.44001509],
        [1.19192944, 0.57572052],
        [1.13670741, 0.63755991],
        [1.23266068, 0.51967709],
        [1.29077938, 0.40241388],
        [1.23872101, 0.51054226],
        [1.27420218, 0.44475961],
        [1.1646979 , 0.60931054],
        [1.19971799, 0.56621345],
        [1.26470853, 0.46479454],
        [1.17546308, 0.59566165],
        [1.06999629, 0.69665422],
        [1.28026181, 0.4305066 ],
        [1.18918272, 0.57918711],
        [1.2222771 , 0.53753678],
        [1.15577907, 0.61796597],
        [1.27975085, 0.43221257],
        [1.27268578, 0.44794522],
        [1.19719353, 0.56965389],
        [1.23066388, 0.5253896 ],

In [ ]:
pf, *results = test("dascmop5", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x119851d60>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 34.070485 seconds
minimize_ executed in 29.941836 seconds
minimize_ executed in 30.133125 seconds
compare_on executed in 94.145973 seconds
Generational Distance (GD): (0.18568481474301352, 0.18798807793681016) | (0.18568481474301352, 0.18798807793681016) | (0.18568481474301352, 0.18798807793681016)
Generational Distance Plus (GD+): (0.1856833154826525, 0.18798807793681016) | (0.1856833154826525, 0.18798807793681016) | (0.1856833154826525, 0.18798807793681016)
Inverted Generational Distance (IGD): (0.5028990239771907, 0.46258946576222804) | (0.5028990239771907, 0.46258946576222804) | (0.5028990239771907, 0.46258946576222804)
Inverted Generational Distance Plus (IGD+): (0.44557540180178173, 0.4021815394102295) | (0.44557540180178173, 0.4021815394102295) | (0.44557540180178173, 0.4021815394102295)


(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.6836140e-01],
        [2.0020020e-03, 9.5525626e-01],
        ...,
        [9.9799800e-01, 1.0015025e-03],
        [9.9899900e-01, 5.0062581e-04],
        [1.0000000e+00, 0.0000000e+00]]),
 array([[0.5145535 , 0.52242151],
        [0.34948748, 0.67943601],
        [1.0534047 , 0.18260183],
        [0.48865687, 0.53455247],
        [0.37528487, 0.65666597],
        [0.579401  , 0.47127001],
        [0.5160611 , 0.51966055],
        [1.03592295, 0.19621061],
        [0.62467148, 0.43641909],
        [0.50616003, 0.52403076],
        [0.82935029, 0.31264783],
        [0.30069613, 0.73540509],
        [1.11804637, 0.14622409],
        [0.36097242, 0.67098065],
        [0.28239141, 0.75705681],
        [0.38149395, 0.63440805],
        [0.98012094, 0.22299115],
        [0.17030094, 0.95653455],
        [0.34856878, 0.68097129],
        [0.56317922, 0.48719597],
        [0.28907611, 0.74915026],
        [0.47271188, 0.55436417

In [ ]:
pf, *results = test("dascmop6", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x1285f13d0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.950622 seconds
minimize_ executed in 30.890321 seconds
minimize_ executed in 29.715963 seconds
compare_on executed in 94.557533 seconds
Generational Distance (GD): (0.44307009412144355, 0.4734180427032431) | (0.44307009412144355, 0.4734180427032431) | (0.44307009412144355, 0.4734180427032431)
Generational Distance Plus (GD+): (0.44307009412144355, 0.4734180427032431) | (0.44307009412144355, 0.4734180427032431) | (0.44307009412144355, 0.4734180427032431)
Inverted Generational Distance (IGD): (0.8389992250335994, 0.8679893390105399) | (0.8389992250335994, 0.8679893390105399) | (0.8389992250335994, 0.8679893390105399)
Inverted Generational Distance Plus (IGD+): (0.8155067518234163, 0.8520974385800351) | (0.8155067518234163, 0.8520974385800351) | (0.8155067518234163, 0.8520974385800351)


(array([[0.0000000e+00, 1.0000000e+00],
        [1.0010010e-03, 9.7622292e-01],
        [2.0020020e-03, 9.7097736e-01],
        [3.0030030e-03, 9.6877712e-01],
        [4.0040040e-03, 9.6814945e-01],
        [1.5015015e-01, 9.6522640e-01],
        [1.5115115e-01, 9.5832128e-01],
        [1.5215215e-01, 9.5133460e-01],
        [1.5315315e-01, 9.4426774e-01],
        [1.5415415e-01, 9.3712209e-01],
        [1.5515516e-01, 9.2989905e-01],
        [1.5615616e-01, 9.2260006e-01],
        [1.5715716e-01, 9.1522656e-01],
        [1.5815816e-01, 9.0778004e-01],
        [1.5915916e-01, 9.0026198e-01],
        [1.6016016e-01, 8.9267389e-01],
        [1.6116116e-01, 8.8501730e-01],
        [1.6216216e-01, 8.7729376e-01],
        [1.6316316e-01, 8.6950484e-01],
        [1.6416416e-01, 8.6165211e-01],
        [1.6516517e-01, 8.5373718e-01],
        [1.6616617e-01, 8.4576167e-01],
        [1.6716717e-01, 8.3772721e-01],
        [1.6816817e-01, 8.2963546e-01],
        [1.6916917e-01, 8.2148807e-01],


In [ ]:
pf, *results = test("dascmop7", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x131850460>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 33.784088 seconds
minimize_ executed in 30.273540 seconds
minimize_ executed in 31.455495 seconds
compare_on executed in 95.514163 seconds
Generational Distance (GD): (0.17712190272769768, 0.17531858404043432) | (0.17712190272769768, 0.17531858404043432) | (0.17712190272769768, 0.17531858404043432)
Generational Distance Plus (GD+): (0.17712190272769768, 0.17531858404043432) | (0.17712190272769768, 0.17531858404043432) | (0.17712190272769768, 0.17531858404043432)
Inverted Generational Distance (IGD): (0.6052126872227575, 0.5722747390239891) | (0.6052126872227575, 0.5722747390239891) | (0.6052126872227575, 0.5722747390239891)
Inverted Generational Distance Plus (IGD+): (0.5228883376891581, 0.49661850773137883) | (0.5228883376891581, 0.49661850773137883) | (0.5228883376891581, 0.49661850773137883)


(array([[0.        , 0.09090909, 0.90909091],
        [0.        , 0.1010101 , 0.8989899 ],
        [0.        , 0.11111111, 0.88888889],
        ...,
        [0.88888889, 0.        , 0.11111111],
        [0.8989899 , 0.        , 0.1010101 ],
        [0.90909091, 0.        , 0.09090909]]),
 array([[0.09905932, 0.09940678, 1.09808878],
        [0.08593377, 0.84050857, 0.32312417],
        [1.06977423, 0.08718402, 0.0862831 ],
        ...,
        [0.17000853, 0.31038014, 0.80716642],
        [0.62002066, 0.36274308, 0.35231255],
        [0.38200474, 0.09553606, 0.80574249]]),
 array([[0.09905932, 0.09940678, 1.09808878, 0.        , 0.        ],
        [0.08593377, 0.84050857, 0.32312417, 0.        , 0.        ],
        [1.06977423, 0.08718402, 0.0862831 , 0.        , 0.        ],
        ...,
        [0.17000853, 0.31038014, 0.80716642, 0.        , 0.        ],
        [0.62002066, 0.36274308, 0.35231255, 0.        , 0.        ],
        [0.38200474, 0.09553606, 0.80574249, 0.        

In [ ]:
pf, *results = test("dascmop8", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x128753760>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 36.563372 seconds
minimize_ executed in 32.228053 seconds
minimize_ executed in 31.406443 seconds
compare_on executed in 100.198553 seconds
Generational Distance (GD): (0.10508703994080515, 0.10551630931580162) | (0.10508703994080515, 0.10551630931580162) | (0.10508703994080515, 0.10551630931580162)
Generational Distance Plus (GD+): (0.10506837465979224, 0.10551630931580162) | (0.10506837465979224, 0.10551630931580162) | (0.10506837465979224, 0.10551630931580162)
Inverted Generational Distance (IGD): (0.6754131586651018, 0.6451556264551437) | (0.6754131586651018, 0.6451556264551437) | (0.6754131586651018, 0.6451556264551437)
Inverted Generational Distance Plus (IGD+): (0.5433269420554127, 0.5475406018654245) | (0.5433269420554127, 0.5475406018654245) | (0.5433269420554127, 0.5475406018654245)


(array([[9.9195481e-01, 1.2659245e-01, 0.0000000e+00],
        [9.8982144e-01, 1.4231484e-01, 0.0000000e+00],
        [9.8743889e-01, 1.5800140e-01, 0.0000000e+00],
        ...,
        [4.0165169e-03, 1.2652872e-01, 9.9195481e-01],
        [2.0085113e-03, 1.2657652e-01, 9.9195481e-01],
        [7.7515522e-18, 1.2659245e-01, 9.9195481e-01]]),
 array([[0.07949873, 1.07842602, 0.08606803],
        [0.64545201, 0.8670097 , 0.06054062],
        [1.02439665, 0.06138562, 0.28109949],
        ...,
        [0.31092843, 0.2176382 , 1.0217837 ],
        [0.09859814, 0.13100795, 1.06855867],
        [0.67005159, 0.72950725, 0.49476807]]),
 array([[0.07949873, 1.07842602, 0.08606803, 0.        , 0.        ],
        [0.64545201, 0.8670097 , 0.06054062, 0.        , 0.        ],
        [1.02439665, 0.06138562, 0.28109949, 0.        , 0.        ],
        ...,
        [0.31092843, 0.2176382 , 1.0217837 , 0.        , 0.        ],
        [0.09859814, 0.13100795, 1.06855867, 0.        , 0.        ],
 

In [ ]:
pf, *results = test("dascmop9", 500, 3)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (3,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x1283f05b0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 35.239740 seconds
minimize_ executed in 31.373263 seconds
minimize_ executed in 32.174217 seconds
compare_on executed in 98.788000 seconds
Generational Distance (GD): (0.011593700562430873, 0.006799496055845643) | (0.011593700562430873, 0.006799496055845643) | (0.011593700562430873, 0.006799496055845643)
Generational Distance Plus (GD+): (0.010518065198589225, 0.005407700924470127) | (0.010518065198589225, 0.005407700924470127) | (0.010518065198589225, 0.005407700924470127)
Inverted Generational Distance (IGD): (0.6408971751069383, 0.6324551168635262) | (0.6408971751069383, 0.6324551168635262) | (0.6408971751069383, 0.6324551168635262)
Inverted Generational Distance Plus (IGD+): (0.41678020466217147, 0.40736942930870307) | (0.41678020466217147, 0.40736942930870307) | (0.41678020466217147, 0.40736942930870

(array([[9.9195481e-01, 1.2659245e-01, 0.0000000e+00],
        [9.8982144e-01, 1.4231484e-01, 0.0000000e+00],
        [9.8743889e-01, 1.5800140e-01, 0.0000000e+00],
        ...,
        [4.0165169e-03, 1.2652872e-01, 9.9195481e-01],
        [2.0085113e-03, 1.2657652e-01, 9.9195481e-01],
        [7.7515522e-18, 1.2659245e-01, 9.9195481e-01]]),
 array([[5.15316574e-04, 6.53295689e-01, 7.57789622e-01],
        [1.71345451e-01, 4.67292831e-01, 8.77785939e-01],
        [1.64252511e-01, 1.14171892e+00, 3.75325729e-01],
        ...,
        [7.74991464e-02, 1.26756336e-01, 9.89509198e-01],
        [1.35171192e-01, 3.23488852e-01, 9.38945234e-01],
        [1.06903032e-01, 3.16895837e-01, 9.44990602e-01]]),
 array([[5.15316574e-04, 6.53295689e-01, 7.57789622e-01, 0.00000000e+00,
         0.00000000e+00],
        [1.71345451e-01, 4.67292831e-01, 8.77785939e-01, 0.00000000e+00,
         0.00000000e+00],
        [1.64252511e-01, 1.14171892e+00, 3.75325729e-01, 0.00000000e+00,
         0.00000000e+

# DASCMOP 4

In [ ]:
pf, *results = test("dascmop1", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x128753cd0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 39.839687 seconds
minimize_ executed in 29.722597 seconds
minimize_ executed in 31.677502 seconds
compare_on executed in 101.240451 seconds
Generational Distance (GD): (0.0003556870398065451, 0.0003554176179200717) | (0.0003160187977106179, 0.00029423588654001924) | (0.0003579743074405028, 0.0003444641655278656)
Generational Distance Plus (GD+): (0.00029198747596411925, 0.0002799906927150042) | (0.00025574019721919677, 0.00023722693304123466) | (0.0002900854884590197, 0.00026870796495309657)
Inverted Generational Distance (IGD): (0.4425297730157356, 0.43496386321756514) | (0.4295356516742358, 0.42921511851679156) | (0.43307403767741837, 0.42918004851758507)
Inverted Generational Distance Plus (IGD+): (0.3293304309873457, 0.3284982460777453) | (0.3277617547350455, 0.3276612342670009) | (0.3282264283899155

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.82368842, 1.39529967],
        [0.74692949, 1.43906925],
        [0.80382652, 1.40781566],
        [0.74550419, 1.43988573],
        [0.75001437, 1.43761223],
        [0.80602822, 1.40638545],
        [0.80843942, 1.40487725],
        [0.79193214, 1.415157  ],
        [0.72732755, 1.44851607],
        [0.79215206, 1.41465991],
        [0.74905307, 1.43812833],
        [0.79361304, 1.41385945],
        [0.74471344, 1.44019461],
        [0.79511555, 1.41308978],
        [0.73221449, 1.4461434 ],
        [0.794954  , 1.41318829],
        [0.79928604, 1.41054598],
        [0.79738653, 1.41156187],
        [0.73474609, 1.44496178],
        [0.72671957, 1.44871128],
        [0.7395262 , 1.44269144],
        [0.74397747, 1.44048009],
        [0.74455936, 1.44025507],
        [0.80559257, 1.

In [ ]:
pf, *results = test("dascmop2", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x1286cd070>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.257952 seconds
minimize_ executed in 30.925472 seconds
minimize_ executed in 28.912692 seconds
compare_on executed in 93.096608 seconds
Generational Distance (GD): (0.0003907825023690712, 0.0003822074690869387) | (0.0004480009219624976, 0.0004070175595844713) | (0.00041433106134750275, 0.00037626307575962833)
Generational Distance Plus (GD+): (0.00033937505639242637, 0.000327626074743792) | (0.0003960966231327013, 0.0003535392759420053) | (0.00036281503582300174, 0.00031497557025217393)
Inverted Generational Distance (IGD): (0.36080758737658525, 0.3579284715455353) | (0.35514492682486665, 0.35526912050147) | (0.3626220953774955, 0.35931400272391933)
Inverted Generational Distance Plus (IGD+): (0.21330070204995755, 0.21054397582190243) | (0.210004937985612, 0.2090775276875333) | (0.21418766663295521, 0

(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.82193805, 0.93269189],
        [0.82111692, 0.93333544],
        [0.84282604, 0.91460219],
        [0.74432975, 1.00604532],
        [0.84405823, 0.91361988],
        [0.82141557, 0.93311718],
        [0.82186972, 0.93273613],
        [0.82367777, 0.93107821],
        [0.83441345, 0.92175445],
        [0.82202611, 0.93253114],
        [0.74712849, 1.00329167],
        [0.80898098, 0.94416875],
        [0.83065444, 0.92530826],
        [0.84253461, 0.91476044],
        [0.82283635, 0.93196206],
        [0.83069553, 0.92497711],
        [0.80708441, 0.94591544],
        [0.85021777, 0.90849539],
        [0.80551979, 0.9473101 ],
        [0.82931684, 0.92619943],
        [0.79622922, 0.95580605],
        [0.80497492, 0.9478087 ],
        [0.89779607, 0.86961042],

In [ ]:
pf, *results = test("dascmop3", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x1285e0640>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 31.004890 seconds
minimize_ executed in 28.885943 seconds
minimize_ executed in 29.542331 seconds
compare_on executed in 89.433758 seconds
Generational Distance (GD): (0.14749805621133733, 0.195742789797537) | (0.1375648967609783, 0.19070912912336518) | (0.002456412505818402, 0.002442360060581364)
Generational Distance Plus (GD+): (0.13552565968478766, 0.18301405199217669) | (0.12813045246827565, 0.18213307182750194) | (0.0007492158295479954, 0.0005015003242083704)
Inverted Generational Distance (IGD): (0.49779456467722333, 0.5141635203917957) | (0.491967489147815, 0.5074016610282974) | (0.4123550517313609, 0.41160792283581593)
Inverted Generational Distance Plus (IGD+): (0.3865317016539016, 0.42957255037884323) | (0.37501670352503225, 0.4202105723714903) | (0.22138007087705477, 0.22013351719558472)


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.69219219, 1.1227715 ],
        [0.69319319, 1.1138213 ],
        [0.69419419, 1.1048608 ],
        [0.6951952 , 1.095892  ],
        [0.6961962 , 1.0869168 ],
        [0.6971972 , 1.0779371 ],
        [0.6981982 , 1.0689549 ],
        [0.6991992 , 1.0599719 ],
        [0.7002002 , 1.054135  ],
        [0.89239239, 0.93319559],
        [0.89339339, 0.92458428],
        [0.89439439, 0.91596118],
        [0.8953954 , 0.90732822],
        [0.8963964 , 0.89868734],
        [0.8973974 , 0.89004046],
        [0.898

In [ ]:
pf, *results = test("dascmop4", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x1286aaa00>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.140517 seconds
minimize_ executed in 28.887519 seconds
minimize_ executed in 23.317997 seconds
compare_on executed in 84.346551 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[1.32013275, 0.82749288],
        [1.32228793, 0.82469966],
        [1.19261958, 1.02031199],
        [1.4579167 , 0.58265437],
        [1.40620835, 0.68117578],
        [1.3270827 , 0.81661407],
        [1.24486104, 0.94632481],
        [1.15051754, 1.07706285],
        [1.23889553, 0.95432695],
        [1.43983227, 0.61715279],
        [1.24443039, 0.9468597 ],
        [1.21382917, 0.99182266],
        [1.33561447, 0.80187134],
        [1.14707925, 1.08217466],
        [1.31085301, 0.84258051],
        [1.42665859, 0.64296661],
        [1.31914614, 0.83051055],
        [1.33221088, 0.80932906],
        [1.31077739, 0.84308614],
        [0.55277865, 1.49725839],
        [1.45447108, 0.58941248],
        [1.20957401, 0.99682791],
        [1.24801244, 0.94067465],
        [1.30590126, 0.

In [ ]:
pf, *results = test("dascmop5", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x12868d820>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.524987 seconds
minimize_ executed in 26.282796 seconds
minimize_ executed in 21.240215 seconds
compare_on executed in 80.048624 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.81605688, 0.93817452],
        [0.71757931, 1.0353853 ],
        [0.7325918 , 1.01773514],
        [0.62093677, 1.15512748],
        [1.09888062, 0.72746233],
        [0.51446077, 1.38249678],
        [1.32595607, 0.59153504],
        [0.61792703, 1.15877649],
        [0.91145232, 0.85918812],
        [1.3577476 , 0.57423165],
        [0.5428924 , 1.2933312 ],
        [1.2378334 , 0.641081  ],
        [0.53790504, 1.30732526],
        [0.91764394, 0.85679979],
        [0.55396959, 1.27068638],
        [1.21490297, 0.6547969 ],
        [0.92655703, 0.84704145],
        [0.5282065 , 1.33478065],
        [0.71427234, 1.03898824],
        [0.74075235, 1.01008636],
        [0.84877837, 0.91052099],
        [0.9929298 , 0.79795302],
        [0.8114681 , 0.9422875 ],

In [ ]:
pf, *results = test("dascmop6", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x1287530a0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 30.592975 seconds
minimize_ executed in 28.669973 seconds
minimize_ executed in 22.660992 seconds
compare_on executed in 81.924462 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.69219219, 1.1227715 ],
        [0.69319319, 1.1138213 ],
        [0.69419419, 1.1048608 ],
        [0.6951952 , 1.095892  ],
        [0.6961962 , 1.0869168 ],
        [0.6971972 , 1.0779371 ],
        [0.6981982 , 1.0689549 ],
        [0.6991992 , 1.0599719 ],
        [0.7002002 , 1.054135  ],
        [0.89239239, 0.93319559],
        [0.89339339, 0.92458428],
        [0.89439439, 0.91596118],
        [0.8953954 , 0.90732822],
        [0.8963964 , 0.89868734],
        [0.8973974 , 0.89004046],
        [0.898

In [ ]:
pf, *results = test("dascmop7", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x128f3ee20>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 32.480000 seconds
minimize_ executed in 29.539722 seconds
minimize_ executed in 23.550699 seconds
compare_on executed in 85.570963 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.51010101, 1.489899  ],
        [0.5       , 0.52020202, 1.479798  ],
        ...,
        [1.479798  , 0.5       , 0.52020202],
        [1.489899  , 0.5       , 0.51010101],
        [1.5       , 0.5       , 0.5       ]]),
 array([[1.50039356, 0.50092633, 0.50092844],
        [0.5084436 , 1.49115514, 0.50370317],
        [0.5006303 , 0.50042204, 1.49925631],
        ...,
        [1.01449404, 0.77306032, 0.73199854],
        [0.78565106, 0.53269147, 1.18850363],
        [1.39384024, 0.51780097, 0.62916951]]),
 array([[1.49723729, 0.50332347, 0.50920168, 0.        , 0.        ],
        [0.50106607, 0.5058689 , 1.49623716, 0.        , 0.        ],
        [0.51795749, 0.50142597, 1.4808716 , 0.        , 0.        ],
        ...,
        [1.31589269, 0.65972724, 0.53335751, 0.        , 0.        ],
        [0.95788187, 0.93899674, 0.60991254, 0.        , 0.        ],
        [0.53747419, 1.45589884, 0.5100453 , 0.        

In [ ]:
pf, *results = test("dascmop8", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x1285b2d00>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 33.333141 seconds
minimize_ executed in 29.057088 seconds
minimize_ executed in 23.340988 seconds
compare_on executed in 85.731704 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[1.48436969, 0.50049077, 0.67716515],
        [0.50024865, 0.50301973, 1.50010395],
        [0.50089881, 0.50212124, 1.50083996],
        ...,
        [1.32955392, 0.94214651, 0.85304459],
        [1.288518  , 0.52450613, 1.12219674],
        [1.10671492, 1.13326465, 0.98923511]]),
 array([[0.50261869, 0.50197675, 1.50117818, 0.        , 0.        ],
        [1.50049286, 0.50145353, 0.50053831, 0.        , 0.        ],
        [1.31997087, 1.0724149 , 0.50004896, 0.        , 0.        ],
        ...,
        [1.26055841, 0.52786566, 1.15528175, 0.        , 0.        ],
        [1.47625875, 0.65680196, 0.67877685, 0.        , 0.        ],
        [0.92147549, 0.92496231, 1.31043888, 0.        

In [ ]:
pf, *results = test("dascmop9", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x128546130>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 32.267229 seconds
minimize_ executed in 29.272012 seconds
minimize_ executed in 29.236267 seconds
compare_on executed in 90.776082 seconds
Generational Distance (GD): (0.005574046901285079, 0.005479457711344652) | (0.005638264102720504, 0.005486174814596772) | (0.005770016191012888, 0.005779715342435808)
Generational Distance Plus (GD+): (0.004168716115726188, 0.0037054250370208708) | (0.004144176335979361, 0.0035447120297805723) | (0.004148928027218507, 0.0038048168716435724)
Inverted Generational Distance (IGD): (0.6164842109748915, 0.6028470570174316) | (0.6060857648665073, 0.5949451866614919) | (0.6020992421018433, 0.5940743137244371)
Inverted Generational Distance Plus (IGD+): (0.4332924476242349, 0.4239439598474637) | (0.42734355014367603, 0.4163564838267298) | (0.4273283609823944, 0.416126137898242

(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.90685857, 1.26700782, 1.02693882],
        [0.50017568, 0.79486979, 1.45561192],
        [0.51498783, 1.50023306, 0.56841598],
        ...,
        [0.68076357, 1.05988533, 1.30900121],
        [0.76282439, 1.08314373, 1.26882685],
        [0.55077727, 0.87392498, 1.42709143]]),
 array([[0.50275075, 0.50740389, 1.50021196, 0.        , 0.        ],
        [0.50008164, 0.58012894, 1.4968171 , 0.        , 0.        ],
        [0.51708585, 1.45668726, 0.81472089, 0.        , 0.        ],
        ...,
        [0.71501837, 1.1406973 , 1.2376535 , 0.        , 0.        ],
        [0.61498714, 0.72051926, 1.47023357, 0.        , 0.        ],
        [0.59284702, 1.29289433, 1.10263368, 0.        

# DASCMOP 5

In [ ]:
pf, *results = test("dascmop1", 500, 5)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (5,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x1285e0640>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.207217 seconds
minimize_ executed in 31.670922 seconds
minimize_ executed in 33.249419 seconds
compare_on executed in 100.128321 seconds
Generational Distance (GD): (0.00040630323368679973, 0.00035839312963238115) | (0.00040370469828418607, 0.0003541731425896735) | (0.00040370469828418607, 0.0003541731425896735)
Generational Distance Plus (GD+): (0.00035412855660320926, 0.00029495834062776005) | (0.00035767967306190093, 0.00030160322271786293) | (0.00035767967306190093, 0.00030160322271786293)
Inverted Generational Distance (IGD): (0.37887136502331, 0.3786087211556472) | (0.3777657689059739, 0.3778949491979246) | (0.3777657689059739, 0.3778949491979246)
Inverted Generational Distance Plus (IGD+): (0.29077320432848247, 0.290612826853629) | (0.29056527227397017, 0.2905040070792242) | (0.2905652722739701

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.94894895, 0.09949589],
        [0.94994995, 0.09759509],
        [1.        , 0.        ]]),
 array([[0.31229173, 0.90854135],
        [0.34546338, 0.88400713],
        [0.31235356, 0.90805491],
        [0.31441377, 0.90342272],
        [0.34130956, 0.88545613],
        [0.33834047, 0.8864571 ],
        [0.34206032, 0.88514317],
        [0.34466361, 0.88490575],
        [0.33766447, 0.88676216],
        [0.34473334, 0.88421974],
        [0.31478246, 0.90295492],
        [0.31528273, 0.90234438],
        [0.3155877 , 0.90154396],
        [0.31737758, 0.90024299],
        [0.31528822, 0.90220865],
        [0.33709256, 0.88711948],
        [0.31740562, 0.89980649],
        [0.31770808, 0.89954914],
        [0.31451074, 0.90319465],
        [0.33650856, 0.88757823],
        [0.33466293, 0.88830951],
        [0.31676064, 0.90041602],
        [0.33271447, 0.88955206],

In [ ]:
pf, *results = test("dascmop1", 500, 6)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (6,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x1285e4e80>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 36.513321 seconds
minimize_ executed in 29.132304 seconds
minimize_ executed in 27.788704 seconds
compare_on executed in 93.435021 seconds
Generational Distance (GD): (0.0003896841963297798, 0.0003660162474315054) | (0.00039630092812079765, 0.0003877615993521002) | (0.00039630092812079765, 0.0003877615993521002)
Generational Distance Plus (GD+): (0.0003436169548438371, 0.00031160396543650215) | (0.0003438083050930045, 0.00032342125578383655) | (0.0003438083050930045, 0.00032342125578383655)
Inverted Generational Distance (IGD): (0.4007171600595103, 0.39839878657496036) | (0.406754702319969, 0.4054359067601392) | (0.406754702319969, 0.4054359067601392)
Inverted Generational Distance Plus (IGD+): (0.3045271265126768, 0.30406342699716954) | (0.30619167265471, 0.30600391915749914) | (0.30619167265471, 0.3060

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.82953996, 1.39200682],
        [0.82310006, 1.3956856 ],
        [0.82165498, 1.39675812],
        [0.76571193, 1.42954177],
        [0.83360121, 1.38878219],
        [0.83196167, 1.39005703],
        [0.78830616, 1.41693776],
        [0.82672966, 1.39346242],
        [0.82584002, 1.39399947],
        [0.82724111, 1.39293837],
        [0.8165356 , 1.39992853],
        [0.80817432, 1.40506922],
        [0.80371403, 1.40776704],
        [0.82316677, 1.39558229],
        [0.85853433, 1.37145655],
        [0.80414674, 1.40767796],
        [0.84649698, 1.3799657 ],
        [0.80791702, 1.40518846],
        [0.86519771, 1.36677358],
        [0.81127467, 1.40318649],
        [0.86785547, 1.36473581],
        [0.76976434, 1.4273776 ],
        [0.82659873, 1.3937196 ],
        [0.80950001, 1.

In [ ]:
pf, *results = test("dascmop1", 500, 7)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (7,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x1318fadc0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.793898 seconds
minimize_ executed in 30.532933 seconds
minimize_ executed in 30.813744 seconds
compare_on executed in 97.141302 seconds
Generational Distance (GD): (0.0003800550138108663, 0.0003619658297349381) | (0.0003800550138108663, 0.0003619658297349381) | (0.0003800550138108663, 0.0003619658297349381)
Generational Distance Plus (GD+): (0.00032380424525261554, 0.00028713387635317977) | (0.00032380424525261554, 0.00028713387635317977) | (0.00032380424525261554, 0.00028713387635317977)
Inverted Generational Distance (IGD): (0.3245513374081239, 0.3244878411933325) | (0.3245513374081239, 0.3244878411933325) | (0.3245513374081239, 0.3244878411933325)
Inverted Generational Distance Plus (IGD+): (0.25593667017600263, 0.2558815893433345) | (0.25593667017600263, 0.2558815893433345) | (0.25593667017600263,

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[0.31269809, 0.90584266],
        [0.3398424 , 0.88806085],
        [0.33559833, 0.88874878],
        [0.31314333, 0.90463814],
        [0.31405886, 0.90331276],
        [0.31477582, 0.90290099],
        [0.31489423, 0.9017274 ],
        [0.31300866, 0.90532706],
        [0.33391962, 0.88920532],
        [0.33326747, 0.8896403 ],
        [0.33462292, 0.8890536 ],
        [0.31406774, 0.90300084],
        [0.33157035, 0.89048044],
        [0.33293965, 0.88974039],
        [0.33193709, 0.89026695],
        [0.31568891, 0.90139784],
        [0.31599228, 0.90091497],
        [0.31544551, 0.90155788],
        [0.31639655, 0.9006883 ],
        [0.33253558, 0.88993137],
        [0.33500745, 0.8888202 ],
        [0.3321865 , 0.89013339],
        [0.3166139 , 0.90024822],

In [ ]:
pf, *results = test("dascmop1", 500, 8)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (8,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x1286b0280>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.743836 seconds
minimize_ executed in 28.782823 seconds
minimize_ executed in 27.761156 seconds
compare_on executed in 90.288781 seconds
Generational Distance (GD): (0.5600611483121528, 0.5634174054907533) | (0.5637736132531064, 0.5651211015347055) | (0.5600835651813842, 0.563806684037034)
Generational Distance Plus (GD+): (0.5600611483121528, 0.5634174054907533) | (0.5637736132531064, 0.5651211015347055) | (0.5600835651813842, 0.563806684037034)
Inverted Generational Distance (IGD): (0.8082535333716115, 0.8091580128596527) | (0.7941956972588655, 0.8017698438868842) | (0.8091852206642888, 0.8092480492710861)
Inverted Generational Distance Plus (IGD+): (0.797502653000016, 0.7988869372106394) | (0.7861839224198, 0.7930149226056775) | (0.7982721456521185, 0.7989710385865045)


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [ ]:
pf, *results = test("dascmop1", 500, 9)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (9,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x128eae100>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 34.959735 seconds
minimize_ executed in 27.062159 seconds
minimize_ executed in 31.316339 seconds
compare_on executed in 93.339038 seconds
Generational Distance (GD): (0.0003483672401223666, 0.0003344445891038013) | (0.000310784500237706, 0.0003002944962222713) | (0.000310784500237706, 0.0003002944962222713)
Generational Distance Plus (GD+): (0.00029022193151945495, 0.0002708946148059358) | (0.00023982505675096074, 0.00022930018475919578) | (0.00023982505675096074, 0.00022930018475919578)
Inverted Generational Distance (IGD): (0.37823876512978216, 0.3782512588636907) | (0.37996350221748754, 0.3800419554192179) | (0.37996350221748754, 0.3800419554192179)
Inverted Generational Distance Plus (IGD+): (0.2879562051060003, 0.2878921715248107) | (0.28836480965122946, 0.28835350020760975) | (0.28836480965122946,

(array([[0.00900901, 0.99991884],
        [0.01001001, 0.9998998 ],
        [0.01101101, 0.99987876],
        [0.01201201, 0.99985571],
        [0.01301301, 0.99983066],
        [0.01401401, 0.99980361],
        [0.01501501, 0.99977455],
        [0.01601602, 0.99974349],
        [0.01701702, 0.99971042],
        [0.01801802, 0.99967535],
        [0.01901902, 0.99963828],
        [0.02002002, 0.9995992 ],
        [0.02102102, 0.99955812],
        [0.02202202, 0.99951503],
        [0.02302302, 0.99946994],
        [0.02402402, 0.99942285],
        [0.02502502, 0.99937375],
        [0.02602603, 0.99932265],
        [0.02702703, 0.99926954],
        [0.02802803, 0.99921443],
        [0.02902903, 0.99915732],
        [0.03003003, 0.9990982 ],
        [0.03103103, 0.99903708],
        [0.03203203, 0.99897395],
        [0.03303303, 0.99890882],
        [0.03403403, 0.99884168],
        [0.03503503, 0.99877255],
        [0.03603604, 0.9987014 ],
        [0.03703704, 0.99862826],
        [0.038

In [ ]:
pf, *results = test("dascmop1", 500, 10)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (10,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x1287ae1c0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.731076 seconds
minimize_ executed in 29.556468 seconds
minimize_ executed in 29.904452 seconds
compare_on executed in 92.192581 seconds
Generational Distance (GD): (0.0003668361806386893, 0.000353331191273738) | (0.00038124035217707177, 0.00035041616446419756) | (0.00038124035217707177, 0.00035041616446419756)
Generational Distance Plus (GD+): (0.00031977563903037013, 0.0003026516031533452) | (0.00033521090396641513, 0.00030207887747108776) | (0.00033521090396641513, 0.00030207887747108776)
Inverted Generational Distance (IGD): (0.3989891011757778, 0.3981220344260037) | (0.40416811246327816, 0.4043247649375712) | (0.40416811246327816, 0.4043247649375712)
Inverted Generational Distance Plus (IGD+): (0.3040531493323313, 0.30390986045019347) | (0.30545613621283596, 0.30561734232523213) | (0.305456136212

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.81989364, 1.39806691],
        [0.78260406, 1.42023652],
        [0.75088644, 1.43749587],
        [0.80712336, 1.40567764],
        [0.84510475, 1.38113351],
        [0.87179706, 1.36218456],
        [0.88047853, 1.35613735],
        [0.87603014, 1.35874972],
        [0.7899442 , 1.41594411],
        [0.82879737, 1.39196467],
        [0.8143627 , 1.4015697 ],
        [0.81567128, 1.40074217],
        [0.84028673, 1.38433884],
        [0.84025523, 1.38444594],
        [0.81095944, 1.40367   ],
        [0.79225314, 1.41489043],
        [0.87374663, 1.36050215],
        [0.80305545, 1.4082581 ],
        [0.81770773, 1.39912528],
        [0.83263562, 1.38946162],
        [0.88131683, 1.35465385],
        [0.81629513, 1.40004701],
        [0.87223308, 1.36159085],
        [0.79892957, 1.

In [ ]:
pf, *results = test("dascmop1", 500, 11)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (11,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x131860880>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.795917 seconds
minimize_ executed in 31.273119 seconds
minimize_ executed in 26.413157 seconds
compare_on executed in 93.483020 seconds
Generational Distance (GD): (0.00042496216300023373, 0.0003540382959134587) | (0.00042496216300023373, 0.0003540382959134587) | (0.00042496216300023373, 0.0003540382959134587)
Generational Distance Plus (GD+): (0.0003782309211442391, 0.00030629608181123623) | (0.0003782309211442391, 0.00030629608181123623) | (0.0003782309211442391, 0.00030629608181123623)
Inverted Generational Distance (IGD): (0.2606770604750384, 0.26055633040213627) | (0.2606770604750384, 0.26055633040213627) | (0.2606770604750384, 0.26055633040213627)
Inverted Generational Distance Plus (IGD+): (0.21857219798106986, 0.21842021151143104) | (0.21857219798106986, 0.21842021151143104) | (0.218572197981

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[0.3373735 , 0.8925723 ],
        [0.36969294, 0.86789149],
        [0.33784429, 0.89000452],
        [0.36545098, 0.86853781],
        [0.36846128, 0.86816135],
        [0.33770582, 0.89212255],
        [0.3385381 , 0.88801234],
        [0.3635183 , 0.86901768],
        [0.34072193, 0.88460457],
        [0.33930371, 0.88709698],
        [0.33960815, 0.8860998 ],
        [0.33909215, 0.8873181 ],
        [0.34040589, 0.88546198],
        [0.36928766, 0.86790046],
        [0.33867668, 0.88790528],
        [0.33979266, 0.88564452],
        [0.34119529, 0.88443763],
        [0.36316543, 0.86932386],
        [0.36272964, 0.86959575],
        [0.34028433, 0.88549474],
        [0.34159657, 0.88390984],
        [0.34274301, 0.88288501],
        [0.36179534, 0.869886  ],

In [ ]:
pf, *results = test("dascmop1", 500, 12)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (12,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x13187d7c0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 26.761167 seconds
minimize_ executed in 25.294408 seconds
minimize_ executed in 25.531494 seconds
compare_on executed in 77.587641 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.52902903, 1.4991573 ],
        [0.53003003, 1.4990982 ],
        [0.53103103, 1.4990371 ],
        [0.53203203, 1.4989739 ],
        [0.53303303, 1.4989088 ],
        [0.53403403, 1.4988417 ],
        [0.53503504, 1.4987725 ],
        [0.53603604, 1.4987014 ],
        [0.53703704, 1.4986283 ],
        [0.538

In [ ]:
pf, *results = test("dascmop1", 500, 13)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (13,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x128e8fa30>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 30.921831 seconds
minimize_ executed in 26.805723 seconds
minimize_ executed in 27.042784 seconds
compare_on executed in 84.771176 seconds
Generational Distance (GD): (0.0003309622272543898, 0.0003150111685871954) | (0.000315029564214432, 0.0002854283441396768) | (0.000315029564214432, 0.0002854283441396768)
Generational Distance Plus (GD+): (0.00011802329532818723, 0.00010793147684384774) | (0.0001453763355318385, 0.00013840108657159345) | (0.0001453763355318385, 0.00013840108657159345)
Inverted Generational Distance (IGD): (0.3999804562132814, 0.4001673965516553) | (0.407039433668768, 0.4080257785476228) | (0.407039433668768, 0.4080257785476228)
Inverted Generational Distance Plus (IGD+): (0.3042876940456254, 0.30433830101123005) | (0.3062172556757226, 0.3065065845760472) | (0.3062172556757226, 0.3065

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.79617381, 1.41220291],
        [0.88270992, 1.3534863 ],
        [0.85071822, 1.3769737 ],
        [0.84406951, 1.38165104],
        [0.83601455, 1.38724174],
        [0.84121557, 1.38345419],
        [0.79755591, 1.41133244],
        [0.80182829, 1.40876581],
        [0.80385164, 1.4075347 ],
        [0.83430509, 1.38817875],
        [0.80753397, 1.40526402],
        [0.81076296, 1.40332473],
        [0.83206889, 1.38959373],
        [0.84223901, 1.38279134],
        [0.80911602, 1.40430839],
        [0.80312089, 1.40795858],
        [0.81196318, 1.40258444],
        [0.79641436, 1.41208151],
        [0.83185059, 1.39000628],
        [0.80238375, 1.40842392],
        [0.82617136, 1.39359753],
        [0.82726378, 1.3930131 ],
        [0.82332472, 1.39533696],
        [0.82243077, 1.

In [ ]:
pf, *results = test("dascmop1", 500, 14)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (14,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x12878aac0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 28.833384 seconds
minimize_ executed in 26.480333 seconds
minimize_ executed in 25.347962 seconds
compare_on executed in 80.662471 seconds
Generational Distance (GD): (0.00033532922607931487, 0.00032432254833272465) | (0.0003289666512036063, 0.0003487879402567813) | (0.00029343279781675026, 0.00021627637976539892)
Generational Distance Plus (GD+): (0.00011300078152370018, 0.00010206017268132861) | (0.0001647511937093704, 0.00015065875389391747) | (0.00013311648862163666, 0.00010526393746979057)
Inverted Generational Distance (IGD): (0.38020991409527155, 0.3803676565440617) | (0.3816826059142244, 0.3814213559332113) | (0.37699759593169957, 0.3745358959304403)
Inverted Generational Distance Plus (IGD+): (0.2908299211438907, 0.2908400730851265) | (0.29125261839363487, 0.2910962402441837) | (0.2903042943643

(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        ...,
        [1.4489489 , 0.59949589],
        [1.4499499 , 0.59759509],
        [1.5       , 0.5       ]]),
 array([[0.80326541, 1.40798804],
        [0.84974092, 1.37752145],
        [0.82781248, 1.39265459],
        [0.8130302 , 1.40185229],
        [0.832564  , 1.38929085],
        [0.8090455 , 1.40440869],
        [0.84233511, 1.38271409],
        [0.84541422, 1.38076883],
        [0.83735148, 1.3860858 ],
        [0.83910145, 1.38497921],
        [0.82655553, 1.39320728],
        [0.80492018, 1.40699516],
        [0.80665779, 1.40584719],
        [0.81755757, 1.39907956],
        [0.84722595, 1.37942611],
        [0.83633755, 1.38684707],
        [0.84947484, 1.37778032],
        [0.80559494, 1.40658233],
        [0.84115052, 1.38356855],
        [0.81430516, 1.40111698],
        [0.84833082, 1.37867345],
        [0.81566063, 1.40019844],
        [0.81473426, 1.40080643],

# DASCMOP 15


In [ ]:
pf, *results = test("dascmop1", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x128e7e340>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 26.173476 seconds
minimize_ executed in 24.770751 seconds
minimize_ executed in 22.893552 seconds
compare_on executed in 73.838629 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 None,
 None,
 None)

In [ ]:
pf, *results = test("dascmop2", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x1314883d0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 31.737005 seconds
minimize_ executed in 28.457189 seconds
minimize_ executed in 24.178387 seconds
compare_on executed in 84.373222 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.74914354, 1.00066114],
        [0.8220699 , 0.93234376],
        [0.80524738, 0.94734038],
        [0.83562018, 0.92050585],
        [0.81427976, 0.93929127],
        [0.82605326, 0.92891971],
        [0.83648605, 0.92006818],
        [0.82799377, 0.92713741],
        [0.81613879, 0.9376667 ],
        [0.8157869 , 0.93797334],
        [0.81539938, 0.93844684],
        [0.81581528, 0.93783713],
        [0.82681112, 0.92819026],
        [0.83679388, 0.91952799],
        [0.8360977 , 0.92020675],
        [0.83610062, 0.9201119 ],
        [0.83204043, 0.92364954],
        [0.80793036, 0.94495219],
        [0.80635107, 0.94639005],
        [0.82826789, 0.92694384],
        [0.80524061, 0.94734324],
        [0.81627023, 0.9376062 ],
        [0.81515935, 0.93851847],

In [ ]:
pf, *results = test("dascmop3", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x128f80040>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.906086 seconds
minimize_ executed in 28.712768 seconds
minimize_ executed in 23.210904 seconds
compare_on executed in 84.830495 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [ ]:
pf, *results = test("dascmop3", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x128753cd0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 29.737236 seconds
minimize_ executed in 26.670827 seconds
minimize_ executed in 22.245836 seconds
compare_on executed in 78.654568 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [ ]:
pf, *results = test("dascmop4", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x1290bad30>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 27.651410 seconds
minimize_ executed in 27.619215 seconds
minimize_ executed in 24.363525 seconds
compare_on executed in 79.634820 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.50538852, 1.49990456],
        [1.49985756, 0.50000391],
        [0.65793661, 1.47503824],
        [1.14240347, 1.08708975],
        [0.53670896, 1.49868462],
        [0.647593  , 1.47822048],
        [0.52150064, 1.4995766 ],
        [0.56211632, 1.49614721],
        [0.58537105, 1.49268606],
        [0.55313595, 1.49721781],
        [0.57627618, 1.49424675],
        [0.58835027, 1.49213993],
        [0.56750043, 1.49533543],
        [0.51124475, 1.49980844],
        [0.54955075, 1.49749007],
        [0.6183992 , 1.48587675],
        [0.62842231, 1.4835989 ],
        [0.60343307, 1.48926499],
        [0.59576551, 1.49088223],
        [0.60891911, 1.48823173],
        [0.61408716, 1.48701307],
        [0.51590803, 1.49972752],
        [0.54355328, 1.49815948],
        [1.31322087, 0.

In [ ]:
pf, *results = test("dascmop5", 1000, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x1286024f0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 60.131689 seconds
minimize_ executed in 55.992424 seconds
minimize_ executed in 50.226597 seconds
compare_on executed in 166.351380 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.8382888 , 0.91837373],
        [0.49990839, 1.49700402],
        [0.52146856, 1.35304921],
        [0.61118405, 1.16632563],
        [0.50007177, 1.48735911],
        [0.50421805, 1.43454129],
        [0.82418319, 0.93044939],
        [0.51982078, 1.35882955],
        [0.80876957, 0.94425543],
        [0.53326338, 1.31729973],
        [0.51894483, 1.36195862],
        [0.52235397, 1.35014471],
        [0.52289019, 1.34836774],
        [0.72927108, 1.0210589 ],
        [0.67366776, 1.08305214],
        [0.58669236, 1.20541874],
        [0.75594993, 0.99395197],
        [0.65228379, 1.10954804],
        [0.58806205, 1.20298031],
        [0.74637903, 1.00345677],
        [0.55312528, 1.26921595],
        [0.53421883, 1.31469084],
        [0.50467169, 1.43129833],

In [ ]:
pf, *results = test("dascmop6", 1000, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x128f800a0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 63.349674 seconds
minimize_ executed in 54.088596 seconds
minimize_ executed in 49.981097 seconds
compare_on executed in 167.420181 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

# DASCMOP 16

In [ ]:
pf, *results = test("dascmop1", 1000, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x7c1662ec6ad0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 101.832141 seconds
minimize_ executed in 104.066108 seconds
minimize_ executed in 96.817233 seconds
compare_on executed in 302.715875 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [ ]:
pf, *results = test("dascmop2", 1000, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x7c166323cfa0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 123.697541 seconds
minimize_ executed in 109.284381 seconds
minimize_ executed in 96.467283 seconds
compare_on executed in 329.449517 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.4489489 , 0.52585989],
        [1.4499499 , 0.52534624],
        [1.5       , 0.5       ]]),
 array([[0.83839984, 0.91837828],
        [0.80207923, 0.9503406 ],
        [0.80546628, 0.94723174],
        ...,
        [0.8371347 , 0.91918969],
        [0.83081073, 0.92465375],
        [0.83407032, 0.92182686]]),
 array([[0.80398512, 0.94848886, 0.        , 0.        ],
        [0.80883469, 0.94408968, 0.        , 0.        ],
        [0.80618372, 0.94649752, 0.        , 0.        ],
        ...,
        [0.83198802, 0.92376359, 0.        , 0.        ],
        [0.80406343, 0.94843667, 0.        , 0.        ],
        [0.80457797, 0.94793957, 0.        , 0.        ]]),
 None)

In [ ]:
pf, *results = test("dascmop3", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x7c1663459840>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 54.205597 seconds
minimize_ executed in 53.474943 seconds
minimize_ executed in 47.532207 seconds
compare_on executed in 155.213127 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.7002002 , 1.054135  ],
        [0.9044044 , 0.89863654],
        [1.1066066 , 0.77294554],
        [1.3008008 , 0.61141457],
        [1.5       , 0.5       ]]),
 array([[0.84946302, 1.26547234],
        [0.84981388, 1.26355107],
        [0.84995667, 1.26217475],
        [0.90389637, 0.89561829]]),
 array([[0.8495091 , 1.26443892, 0.        , 0.        ],
        [0.84938013, 1.26533869, 0.        , 0.        ],
        [0.90387847, 0.89553342, 0.        , 0.        ]]),
 None)

In [ ]:
pf, *results = test("dascmop4", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x7c1663462b30>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 52.581327 seconds
minimize_ executed in 52.821261 seconds
minimize_ executed in 46.190309 seconds
compare_on executed in 151.593217 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [ ]:
pf, *results = test("dascmop5", 200, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x7c1662d5e290>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 8.794743 seconds
minimize_ executed in 8.042573 seconds
minimize_ executed in 7.686258 seconds
compare_on executed in 24.523950 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        [0.503003  , 1.4452003 ],
        [0.504004  , 1.4367228 ],
        [0.50500501, 1.4292539 ],
        [0.50600601, 1.4225016 ],
        [0.50700701, 1.4162921 ],
        [0.50800801, 1.4105125 ],
        [0.50900901, 1.4050842 ],
        [0.51001001, 1.39995   ],
        [0.51101101, 1.3950666 ],
        [0.51201201, 1.3904007 ],
        [0.51301301, 1.3859254 ],
        [0.51401401, 1.3816192 ],
        [0.51501502, 1.3774642 ],
        [0.51601602, 1.3734456 ],
        [0.51701702, 1.3695507 ],
        [0.51801802, 1.3657688 ],
        [0.51901902, 1.3620905 ],
        [0.52002002, 1.3585079 ],
        [0.52102102, 1.3550137 ],
        [0.52202202, 1.3516018 ],
        [0.52302302, 1.3482666 ],
        [0.52402402, 1.3450031 ],
        [0.52502503, 1.341807  ],
        [0.52602603, 1.3386742 ],
        [0.52702703, 1.335601  ],
        [0.52802803, 1.3325843 ],
        [0.529

In [ ]:
pf, *results = test("dascmop6", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x7c1663427f40>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 7.943926 seconds
minimize_ executed in 8.099066 seconds
minimize_ executed in 8.052566 seconds
compare_on executed in 24.095895 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.7002002 , 1.054135  ],
        [0.9044044 , 0.89863654],
        [1.1066066 , 0.77294554],
        [1.3008008 , 0.61141457],
        [1.5       , 0.5       ]]),
 array([[0.50159012, 1.47259605],
        [0.50247126, 1.46952118],
        [0.50135453, 1.47344314],
        [0.50284316, 1.46880979],
        [0.50342231, 1.46820125],
        [0.50113907, 1.47459644],
        [0.50339919, 1.46823783],
        [0.50310499, 1.46854207],
        [0.50219837, 1.47036095],
        [0.50085974, 1.47710368],
        [0.50209312, 1.4705986 ],
        [0.50233524, 1.47021586],
        [0.50125172, 1.47401204],
        [0.50396646, 1.46819546],
        [0.50236109, 1.46989519]]),
 None,
 None)

In [ ]:
pf, *results = test("dascmop7", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x7c1663458850>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 23.231211 seconds
minimize_ executed in 22.582301 seconds
minimize_ executed in 19.719371 seconds
compare_on executed in 65.533279 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.51010101, 1.489899  ],
        [0.5       , 0.52020202, 1.479798  ],
        ...,
        [1.479798  , 0.5       , 0.52020202],
        [1.489899  , 0.5       , 0.51010101],
        [1.5       , 0.5       , 0.5       ]]),
 array([[0.49995151, 0.58086338, 1.41894739],
        [0.5007725 , 0.81480673, 1.18433891],
        [0.50456278, 0.81307626, 1.18229661],
        ...,
        [0.50159884, 0.71030093, 1.28800145],
        [0.50635354, 0.68167909, 1.31203756],
        [0.50513362, 0.67656283, 1.31806582]]),
 array([[0.66661083, 0.74562207, 1.08756818, 0.        , 0.        ],
        [0.52944837, 0.56810277, 1.40261273, 0.        , 0.        ],
        [0.65310024, 0.85652203, 0.99061048, 0.        , 0.        ],
        ...,
        [0.587492  , 0.68751322, 1.22471035, 0.        , 0.        ],
        [0.59359367, 0.71595674, 1.19033   , 0.        , 0.        ],
        [0.59083845, 0.69534691, 1.21377543, 0.        

In [ ]:
pf, *results = test("dascmop8", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x7c1662ec6e30>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 22.764735 seconds
minimize_ executed in 20.023026 seconds
minimize_ executed in 18.035995 seconds
compare_on executed in 60.824070 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[1.06810486, 1.32279854, 0.4999094 ],
        [0.52357156, 0.57481521, 1.49682934],
        [1.1868545 , 0.50004091, 1.2266078 ],
        ...,
        [0.87353104, 1.29932517, 0.97038028],
        [0.83365749, 1.1187738 , 1.21101256],
        [1.14304829, 1.1100606 , 0.96259958]]),
 array([[0.62162905, 1.27212581, 1.12387417, 0.        , 0.        ],
        [0.64061487, 1.48925787, 0.54023881, 0.        , 0.        ],
        [1.14848293, 1.19105897, 0.81972021, 0.        , 0.        ],
        ...,
        [0.78358991, 1.3898219 , 0.85742807, 0.        , 0.        ],
        [0.78555983, 1.40179843, 0.82478093, 0.        , 0.        ],
        [0.78279023, 1.38952551, 0.85862906, 0.        

In [ ]:
pf, *results = test("dascmop9", 200, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x7c16627bc640>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 9.033986 seconds
minimize_ executed in 8.722351 seconds
minimize_ executed in 7.376199 seconds
compare_on executed in 25.133273 seconds
Generational Distance (GD): (0.004947633293754378, 0.0054698433863406525) | (0.0042463874864052936, 0.004147832316327425) | (0.004859143721842364, 0.005501264086089613)
Generational Distance Plus (GD+): (0.0018026241073649646, 0.0015411835873532231) | (0.0019302737566316743, 0.0014922795783554044) | (0.002137501491416426, 0.0019528041964639309)
Inverted Generational Distance (IGD): (0.651082283646549, 0.6504151231599302) | (0.6447669253080689, 0.6440995705825114) | (0.6460264465224134, 0.6440264601096894)
Inverted Generational Distance Plus (IGD+): (0.40403651503786725, 0.40306847633613463) | (0.4054698298159435, 0.40321024470517786) | (0.4034535415899421, 0.402992218

(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.54789907, 0.80337469, 1.45165181],
        [0.53673427, 0.60448135, 1.49378019],
        [0.55207257, 0.92922831, 1.40154131],
        [0.55106387, 0.75970233, 1.46445316],
        [0.5797886 , 0.92151174, 1.4031889 ],
        [0.5706621 , 0.60345733, 1.49205112],
        [0.55486002, 0.79123637, 1.45510142],
        [0.54784103, 0.78240899, 1.45803723],
        [0.54665205, 0.78359409, 1.45769313],
        [0.52748518, 0.64461408, 1.48921021],
        [0.56226309, 0.59227588, 1.49370301],
        [0.54881871, 0.76651855, 1.46261582],
        [0.5843784 , 0.94337569, 1.39226734],
        [0.54768415, 0.78246138, 1.45805934],
        [0.54796773, 0.76837097, 1.46200542],
        [0.56683667